<font size=6>PROJECT 1 - YELP REVIEWS -  
    <font size=5>
THIS WORKBOOK REQUESTS REVIEWS FOR VARIOUS LAWYERS / LAW FIRMS ACROSS RANDOM US CITIES
</font>

<font size=4>By: Kelly Kayser, Michael Obermire, Milan Chen, and Jared Case

In [ ]:
import pandas as pd
import json
import numpy as np
import requests
from config import (client_id, api_key)
from pprint import pprint
import pydbgen
from pydbgen import pydbgen
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

<font size=4>The code below creates a dictionary of lists called business_data to hold the values pulled from the Yelp api request.

In [ ]:
business_data={"Business ID":[],
               "Business Name":[],
               "Business Alias":[],
               "Category":[],
               "City":[],
               "State":[],
               "ZipCode":[],
               "Lat":[],
               "Long":[],
               "Review Count":[],
               "Rating":[],
              }
review_data={"Business ID":[],
             "Review":[],
            }

<font size=4>This code below appends real US cities generated from pydbgen to a list called city. This list of cities will be passed through the Yelp api request as {locations}.  

In [ ]:
myDB=pydbgen.pydb()
city= []
for x in range(600):
    x=(myDB.city_real())
    #print (x)
    if x not in city:
        city.append(x)
city_yelp=pd.DataFrame(city)
city_yelp=city_yelp.rename(columns={0:"US Cities"})

<font size=4>The code below calls the api key, url address, and sets the required search criteria {category} to lawyers and the {radius} to 40,000 m from the {location}. 

In [ ]:
api_key=api_key
headers = {"Authorization": (f"Bearer {api_key}")}
url="https://api.yelp.com/v3/businesses/search?"
review_url="https://api.yelp.com/v3/businesses/"
categories="lawyers"
radius = 40000
counter = 1

<font size=4>The code below sets up a loop to pass the list of random cities through the Yelp api in order to obtain a list of lawyers / law firms within a 40,000 m radius of the {location}. The general information for each lawyer / law firm is appended to the dictionary titled business_data. When passing a city through the Yelp api and no busineses are returned, that city is skipped using the try/except function.

In [ ]:
for index, row in city_yelp.iterrows():
    try:
        random_city = row['US Cities']
        print(f"{counter}: {random_city}")
        yelp_response2 = requests.get((f"{url}categories={categories}&location={random_city}&radius={radius}"), headers=headers, verify=False).json()
        #pprint(yelp_response2)
        counter+=1
    except:
        print(f"{counter}: Could not locate any lawyers in {random_city}.")
        #print(yelp_response2)
    #print(len(yelp_response2["businesses"]))
    try:    
        for x in range(len(yelp_response2["businesses"])):
            try:
                business_data["Business ID"].append(yelp_response2["businesses"][x]["id"])
                business_data["Business Name"].append(yelp_response2["businesses"][x]["name"])
                business_data["Business Alias"].append(yelp_response2["businesses"][x]["alias"])
                business_data["Category"].append(yelp_response2["businesses"][x]["categories"][0]["alias"])
                business_data["City"].append(yelp_response2["businesses"][x]["location"]["city"])
                business_data["State"].append(yelp_response2["businesses"][x]["location"]["state"])
                business_data["ZipCode"].append(yelp_response2["businesses"][x]["location"]["zip_code"])
                business_data["Lat"].append(yelp_response2["businesses"][x]["coordinates"]["latitude"])
                business_data["Long"].append(yelp_response2["businesses"][x]["coordinates"]["longitude"])
                business_data["Review Count"].append(yelp_response2["businesses"][x]["review_count"])
                business_data["Rating"].append(yelp_response2["businesses"][x]["rating"])
            except: 
                print(f"     No law offices found in {random_city} using Yelp.com.")
    except:
        print(f"     No law offices found in {random_city} using Yelp.com.")
        counter+=1

<font size=4>Converting the dictionary titled business_data to a dataframe titled graph_data

In [ ]:
graph_data = pd.DataFrame({ key:pd.Series(value) for key, value in business_data.items() })

<font size=4>The code below requests reviews and Yelp star ratings for each lawyer / law firm that had a {Business ID} obtained during the loop above. The {Business ID} is required when peforming the Yelp api request for reviews and Yelp star ratings. The review and Yelp star rating is stored in a dictionary titled review_data. If a company has no reviews associated with it, the company is skipped using the try/except function.  

In [ ]:
for id in business_data["Business ID"]:
    try:
        yelp_response3 = requests.get((f"{review_url}{id}/reviews"), headers=headers, verify=False).json()
        pprint(yelp_response3)
        print(f"Fetching Reviews for {id}.")
    except:
        print(f"No Reviews for {id}.")
    try:
        for x in range(len(yelp_response3["reviews"])):
            try:
                review_data["Business ID"].append(id)
                review_data["Review"].append(yelp_response3["reviews"][x]["text"])
            except:
                print("skipping")
    except:
        print("skipping")

<font size=4>The code below converts the dictionary titled review_data to a dataframe titled graph_review.

In [ ]:
graph_review = pd.DataFrame({ key:pd.Series(value) for key, value in review_data.items() })

<font size=4>The code below creates a column named "Compound" and adds it to the dataframe titled graph_review. When running the sentiment analysis on each review, the sentiment compound result is appended to the dataframe titled graph_review under the column named "Compound". 

In [ ]:
graph_review["Compound"]=""
graph_review.head()

for index, row in graph_review.iterrows():
    results = analyzer.polarity_scores(row["Review"])
    graph_review.loc[index, "Compound"] = results["compound"]

<font size=4>The code below creates a dataframe titled graph_data. This dataframe is the result of merging the general lawyer and law firm information with the lawyer and law firm review information.

In [ ]:
final=graph_review.merge(graph_data, on="Business ID", how="left")

<font size=4>The code below sends the dataframe titled final to a .csv file in order to clean the data prior to performing graphical analysis.

In [ ]:
final.to_csv(path_or_buf="Merged6.csv", sep=',')